In [ ]:
## Preprocessing
trainData = pd.read_csv("C:\\Users\\venka\\Desktop\\Kaggle\\Titanic\\train.csv")
testData = pd.read_csv("C:\\Users\\venka\\Desktop\\Kaggle\\Titanic\\test.csv")
testY = pd.read_csv("C:\\Users\\venka\\Desktop\\Kaggle\\Titanic\\submission.csv")
fullData = [trainData,testData]

In [ ]:
print(trainData.info())
print(trainData.describe())

In [ ]:
trainData[['Pclass','Survived']].groupby('Pclass').mean()

In [ ]:
trainData[['Sex','Survived']].groupby('Sex').mean()

In [ ]:
trainData[['SibSp','Survived']].groupby('SibSp').mean()

In [ ]:
trainData[['Parch','Survived']].groupby('Parch').mean()

In [ ]:
trainData['Age'] = trainData[['Age']].fillna(trainData[['Age']].dropna().mean())
testData['Age'] = testData[['Age']].fillna(testData[['Age']].dropna().mean())

In [ ]:
for dataset in fullData:
    dataset['Sex'] = dataset['Sex'].map({'female':0,'male':1}).astype(int)

In [ ]:
print(testData.info())
print(trainData.info())

In [ ]:
for dataset in fullData:
    dataset['Fare'] = dataset[['Fare']].fillna(dataset[['Fare']].dropna().median())

In [ ]:
trainData[['Embarked']].describe()

In [ ]:
for dataset in fullData:
    dataset['Embarked'] = dataset[['Embarked']].fillna('S')

In [ ]:
print(trainData['Embarked'].head())

In [ ]:
trainData.Embarked = trainData.Embarked.map({'S': 0,'Q': 1,'C': 2})
testData.Embarked = testData.Embarked.map({'S': 0,'Q': 1,'C': 2})

In [ ]:
## feature Selection
trainData = trainData.drop(['PassengerId','Cabin','Ticket','Name'],axis=1)
testData = testData.drop(['PassengerId','Cabin','Ticket','Name'],axis=1)

In [ ]:
trainData.columns

In [ ]:
trainX = trainData.drop(['Survived'],axis=1)

In [ ]:
trainY = trainData['Survived']

In [ ]:
testData.head()

In [ ]:
import seaborn as sns

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression

In [ ]:
classifiers = [
    KNeighborsClassifier(3),
    SVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
	AdaBoostClassifier(),
    GradientBoostingClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis(),
    LogisticRegression()]

In [ ]:
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.1, random_state=0)

In [ ]:
acc_dict = {}
log_cols = ["Classifier", "Accuracy"]
log  = pd.DataFrame(columns=log_cols)

In [ ]:
for trIndex,tsIndex in sss.split(trainX,trainY):
    Xtrain, Xtest = trainX.iloc[trIndex,],trainX.iloc[tsIndex,]
    Ytrain, Ytest = trainY[trIndex],trainY[tsIndex]
    
    for i,clf in enumerate(classifiers):
        name = clf.__class__.__name__
        clf.fit(Xtrain, Ytrain)
        testPred = clf.predict(Xtest)
        acc = accuracy_score(Ytest, testPred)
        if name in acc_dict:
            acc_dict[name] += acc
        else:
            acc_dict[name] = acc
#print(acc_dict)
for i,clf in enumerate(acc_dict):
    acc_dict[clf] = acc_dict[clf]/10.0
    #print(i,clf,acc_dict[clf])
    singleEntry = pd.DataFrame([[clf,acc_dict[clf]]],columns=log_cols)
    log = log.append(singleEntry)
    
print(log)

In [ ]:
plt.xlabel('Accuracy')
plt.title('Classifier Accuracy')

sns.set_color_codes("muted")
sns.barplot(x='Accuracy', y='Classifier', data=log, color="b")